In [139]:
!pip install pefile

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [140]:
import pefile
import sys

def file_offset_to_virtual_address(pe_file_path, file_offset):
    try:
        # Load the PE file
        pe = pefile.PE(pe_file_path)

        # Get the base address, section alignment, and file alignment
        base_address = pe.OPTIONAL_HEADER.ImageBase
        section_alignment = pe.OPTIONAL_HEADER.SectionAlignment
        file_alignment = pe.OPTIONAL_HEADER.FileAlignment

        # Iterate through sections to find the one containing the file offset
        for section in pe.sections:
            # Get section details
            raw_data_offset = section.PointerToRawData
            raw_data_size = section.SizeOfRawData
            virtual_address = section.VirtualAddress

            # Check if the file offset falls within the section's raw data range
            if raw_data_offset <= file_offset < raw_data_offset + raw_data_size:
                # Calculate the offset within the section
                offset_within_section = file_offset - raw_data_offset

                # Calculate the virtual address
                final_address = base_address + virtual_address + offset_within_section
                print(f"Let's compare! PEFile says {hex(pe.get_rva_from_offset(file_offset))}... I say {hex(final_address)}")
                return final_address

        # If no section contains the file offset, return None
        return None

    except Exception as e:
        print(f"Error processing the PE file: {e}")
        return None


def virtual_address_to_file_offset(pe_file_path, virtual_address):
    try:
        # Load the PE file
        pe = pefile.PE(pe_file_path)

        # Get the base address, section alignment, and file alignment
        base_address = pe.OPTIONAL_HEADER.ImageBase
        section_alignment = pe.OPTIONAL_HEADER.SectionAlignment
        file_alignment = pe.OPTIONAL_HEADER.FileAlignment

        # Calculate the relative virtual address (RVA)
        rva = virtual_address - base_address

        # Iterate through sections to find the one containing the RVA
        for section in pe.sections:
            # Get section details
            raw_data_offset = section.PointerToRawData
            raw_data_size = section.SizeOfRawData
            virtual_address_start = section.VirtualAddress
            virtual_address_end = virtual_address_start + section.Misc_VirtualSize

            # Check if the RVA falls within the section's virtual address range
            if virtual_address_start <= rva < virtual_address_end:
                # Calculate the offset within the section
                offset_within_section = rva - virtual_address_start

                # Calculate the file offset
                file_offset = raw_data_offset + offset_within_section
                print(f"Let's compare! PEFile says {pe.get_offset_from_rva(rva)}... I say {file_offset}")
                return file_offset

        # If no section contains the RVA, return None
        return None

    except Exception as e:
        print(f"Error processing the PE file: {e}")
        return None




In [141]:
executables = {
    "anon_call2_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe",
    "anon_call3_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe",
    "virtual7_vs_stripped": "../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe",
    "anon_jump1_vs_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe",
    "anon_jump1_vs_2_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe",
    "switch4_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe",
    "switch5_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe",
    "switch6_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch6_vs-stripped.exe",
}

cfr_mapping = {
    path: path.replace(".exe", "-cfr.json")
    for key, path in executables.items()
}


In [142]:
import json
import re
for exepath, cfrpath in cfr_mapping.items():
    with open(cfrpath, 'r') as f:
        cfr = json.load(f)
    question = cfr['question']
    match = re.search(r"'([^']*)'(?!.*')", question)
    offset = match.group(1)
    print(f"{exepath}: {offset}")
    file_offset_to_virtual_address(exepath, int(offset,16))

../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe: 0x5d1
Let's compare! PEFile says 0x11d1... I say 0x4011d1
../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe: 0x3f4
Let's compare! PEFile says 0x11f4... I say 0x4011f4
../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe: 0x66c0
Let's compare! PEFile says 0x72c0... I say 0x1400072c0
../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe: 0x43e
Let's compare! PEFile says 0x103e... I say 0x40103e
../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe: 0x43e
Let's compare! PEFile says 0x103e... I say 0x40103e
../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe: 0x6570
Let's compare! PEFile says 0x7170... I say 0x140007170
../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe: 0x6568
Let's compare! PEFile says 0x7168... I say 

In [143]:
import json
import re
import pefile
import os
from typing import List, Dict
from dataclasses import dataclass

@dataclass
class Program:
    exe_path: str
    cfr_path: str
    question: str
    file_offset: int        # offset inside file (from the question)
    rva: int                # runtime RVA (for breakpoints)
    ground_truth: List[int] # list of file offsets from CFR JSON
    gt_rvas: List[int] # list of RVA of GT from CFR JSON

# original flat mapping
executables: Dict[str, str] = {
    "anon_call2_vs_stripped":    "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe",
    "anon_call3_vs_stripped":    "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe",
    "virtual7_vs_stripped":      "../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe",
    "anon_jump1_vs_stripped":    "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe",
    "anon_jump1_vs_2_stripped":  "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe",
    "switch4_vs_stripped":       "../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe",
    "switch5_vs_stripped":       "../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe",
    "switch6_vs_stripped":       "../sunbench25/benchmark/indirect_jumps/switch_statements/switch6_vs-stripped.exe",
}

programs: Dict[str, Program] = {}

for name, exe_path in executables.items():
    # derive the CFR JSON path
    cfr_path = exe_path.replace(".exe", "-cfr.json")

    # load the question, extract the quoted offset, and ground truth
    with open(cfr_path, 'r') as f:
        cfr = json.load(f)

    question = cfr.get('question', '')
    m = re.search(r"'([^']*)'(?!.*')", question)
    if not m:
        raise ValueError(f"Could not parse offset from question for {name!r}")
    file_offset = int(m.group(1), 16)

    # parse ground truth offsets
    gt_strs = cfr.get('groundtruth', [])
    ground_truth = [int(off, 16) for off in gt_strs]

    # compute the RVA from file offset
    pe = pefile.PE(exe_path)
    rva = pe.get_rva_from_offset(file_offset)

    gt_rvas = [hex(pe.get_rva_from_offset(gt)) for gt in ground_truth]
    

    # store everything in our nice dict
    programs[name] = Program(
        exe_path    = exe_path,
        cfr_path    = cfr_path,
        question    = question,
        file_offset = file_offset,
        rva         = rva,
        ground_truth= ground_truth,
        gt_rvas     = gt_rvas
    )


In [195]:
import subprocess
import threading
import queue
import time
import os
import logging
import re
import itertools
from typing import Optional, Callable, List, Dict, Tuple

logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

class ProcessTerminationIntercepted(Exception):
    """Raised when debugger hits NtTerminateProcess — process is about to terminate."""
    pass

_PROMPT_RE   = re.compile(r"\d+:\d+\s*> ?$")      # e.g. “0:000> ” or “12:34> ”
_SENTINEL_ID = itertools.count(1)                 # 1,2,3… — unique per command


def _flush_queue(q: queue.Queue) -> None:
    """Remove every line currently waiting in *q* (non-blocking)."""
    try:
        while True:
            q.get_nowait()
    except queue.Empty:
        pass

class CDBWrapper:
    """
    A thin, logging-centric wrapper around Microsoft CDB/WinDbg.
    Uses a per-command sentinel so every helper blocks until the *next*
    debugger prompt – no more manual `wait_for_prompt()` juggling.
    """

    def __init__(self,
                 cdb_path: str | None = None,
                 symbol_cache_path: str | None = r"C:\symbols") -> None:
        self.cdb_path = cdb_path or r"C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe"
        self.symbol_cache_path = symbol_cache_path
        self.process: Optional[subprocess.Popen] = None
        self.process    : Optional[subprocess.Popen] = None
        self.output_queue = queue.Queue()
        self.output_thread: Optional[threading.Thread] = None
        self.error_thread : Optional[threading.Thread] = None
        self.running    = False
        self.output_callback: Optional[Callable[[str], None]] = None
        self.target_exe = None

        logger.debug("Initialized CDBWrapper with cdb_path=%s", self.cdb_path)

    # PROC LIFECYCLE
    def start(self,
              target_exe: str,
              args: List[str] | None = None,
              output_callback: Callable[[str], None] | None = None) -> bool:
        """
        Launch *target_exe* under CDB.  Returns **True** on success.
        Subsequent helpers block until the debugger prompt re-appears.
        """
        if self.running:
            logger.warning("Debugger is already running")
            return False

        symbol_path = f"srv*{self.symbol_cache_path}*https://msdl.microsoft.com/download/symbols"

        self.output_callback = output_callback
        self.target_exe      = target_exe
        logger.debug("Starting debugger for '%s' with args=%s", target_exe, args)

        cmd = [self.cdb_path,
               "-lines",
               "-y", symbol_path,
               "-srcpath", os.path.dirname(target_exe),
               target_exe]
        if args:
            cmd.extend(args)

        try:
            self.process = subprocess.Popen(
                cmd,
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1,           # line-buffered
            )
            self.running = True

            self.output_thread = threading.Thread(
                target=self._drain, args=(self.process.stdout,)
            )
            self.error_thread  = threading.Thread(
                target=self._drain, args=(self.process.stderr,)
            )
            for t in (self.output_thread, self.error_thread):
                t.daemon = True
                t.start()

            logger.info("Debugger started, PID=%s", self.process.pid)
            # initial loader prompt:
            self._wait_for_prompt()
            return True

        except Exception:
            logger.exception("Failed to start debugger")
            return False

    def attach(self,
               pid: int,
               output_callback: Callable[[str], None] | None = None) -> bool:
        """Attach to a running process."""
        if self.running:
            logger.warning("Debugger is already running")
            return False

        self.output_callback = output_callback
        cmd = [self.cdb_path, "-p", str(pid)]

        try:
            self.process = subprocess.Popen(
                cmd,
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1,
            )
            self.running = True
            self.output_thread = threading.Thread(
                target=self._drain, args=(self.process.stdout,), daemon=True
            )
            self.error_thread  = threading.Thread(
                target=self._drain, args=(self.process.stderr,), daemon=True
            )
            self.output_thread.start()
            self.error_thread.start()

            logger.info("Attached to PID=%s", pid)
            self._wait_for_prompt()
            return True

        except Exception:
            logger.exception("Failed to attach debugger")
            return False

    def stop(self) -> None:
        """Gracefully quit the debugger."""
        if not self.running:
            return

        logger.info("Stopping debugger")
        try:
            self.send_command("q", wait=True)
        except Exception:
            pass

        self.running = False
        if self.process and self.process.poll() is None:
            self.process.terminate()
            self.process.wait(timeout=5)

    def __enter__(self):  return self
    def __exit__(self, exc_type, exc_val, exc_tb):  self.stop()

    # IO
    def _drain(self, stream) -> None:
        """Background reader: push every debugger line into *self.output_queue*."""
        while True:
            try:
                line = stream.readline()
                if not line:
                    break
                self.output_queue.put(line)
                if self.output_callback:
                    self.output_callback(line)
            except Exception:
                break

    def send_command(self, command: str, *, wait: bool = True) -> List[str] | None:
        """
        Send *command* to CDB.  If *wait* (default) is True, block until the
        debugger prints the next prompt and return **all** captured lines.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        sentinel = f"###CDB_SENTINEL_{next(_SENTINEL_ID)}###"

        _flush_queue(self.output_queue)                     # purge stale lines
        full_cmd = f"{command}\n.echo {sentinel}\n"
        logger.debug(">>> %s", command)
        self.process.stdin.write(full_cmd)
        self.process.stdin.flush()

        return self._wait_for(sentinel) if wait else None

    def _wait_for(self, sentinel: str, timeout: float = 10.0) -> List[str]:
        """Internal: block until *sentinel* (or fresh prompt) is seen."""
        deadline = time.time() + timeout
        collected: List[str] = []

        while time.time() < deadline:
            try:
                line = self.output_queue.get(timeout=0.25)
                logger.debug(f"<<< {line}")
                collected.append(line)
                if sentinel in line or _PROMPT_RE.search(line.rstrip()):
                    return collected
            except queue.Empty:
                pass

        raise TimeoutError("Timed out waiting for debugger prompt")

    def _wait_for_prompt(self, timeout: float = 10.0) -> None:
        """Convenience wrapper when no sentinel was used (e.g. after start)."""
        _ = self._wait_for("", timeout=timeout)  # empty sentinel → wait for prompt only

# CONVENIENCE
    def lm(self) -> Tuple[List[str], List[str]]:
        """Return a list of loaded module names and their virtual addresses."""
        lines = self.send_command("lm")
        mods = []
        vaddrs = []
        # Regex to reliably capture start address and module name from 'lm' output.
        mod_re = re.compile(r"^\s*([0-9a-fA-F`]+)\s+([0-9a-fA-F`]+)\s+([^\s]+)")

        if not lines:
            logger.warning("`lm` command returned no output.")
            return [], []

        for ln in lines:
            match = mod_re.match(ln)
            if match:
                # group(1) is the start address, e.g., "00007ff6`e4b40000"
                # group(2) is the module name, e.g., "switch6_vs-stripped"
                vaddrs.append(match.group(1).replace("`",""))
                mods.append(match.group(3))
        
        logger.debug("Loaded modules: %s", mods)
        logger.debug("Module vaddrs: %s", vaddrs)
        return mods, vaddrs

    def bp(self, address: str) -> None:
        """Set breakpoint at *address* (RVA or absolute)."""
        self.send_command(f"bp {address}")

    def g(self, timeout: float = 10.0) -> str:
        """
        Run until next event; returns 'BREAKPOINT' or 'TERMINATED'.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        _flush_queue(self.output_queue)
        logger.debug(">>> g (waiting for BP or Terminate)")
        self.process.stdin.write("g\n")
        self.process.stdin.flush()

        deadline = time.time() + timeout
        while time.time() < deadline:
            try:
                line = self.output_queue.get(timeout=0.25)
                logger.debug(f"<<< {line}")
                # Check for the two events we care about
                if "Breakpoint" in line and "hit" in line:
                    return "BREAKPOINT"
                if "ntdll!NtTerminateProcess" in line:
                    raise ProcessTerminationIntercepted("Process terminated during 'g'")
            except queue.Empty:
                # If the process died without a clean exit message, poll() will catch it.
                if self.process.poll() is not None:
                    raise ProcessTerminationIntercepted("Process terminated unexpectedly")
                continue
        
        raise TimeoutError("Timed out waiting for debugger event after 'g'")

    def t(self) -> None:
        """Single-step one instruction."""
        self.send_command("t")

    def p(self) -> None:
        """Single-step over one instruction."""
        self.send_command("p")

    def r(self) -> Dict[str, str]:
        """Return a dict of current register values."""
        out = self.send_command("r")
        regs: Dict[str, str] = {}
        for ln in out:
            for tok in ln.strip().split():
                if '=' in tok:
                    k, v = tok.split('=', 1)
                    regs[k] = v
        return regs

    def setargs(self, args: List[str]) -> None:
        """Queue new CLI args with `.create`."""
        arg_str = " ".join(f'"{a}"' for a in args)
        self.send_command(f'.create {self.target_exe} {arg_str}')

    def restart(self, target_exe: str, args: List[str] | None = None) -> bool:
        """
        Terminates the current debuggee and starts a new one with new arguments,
        reusing the existing CDB session for speed.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        arg_str = " ".join(f'"{a}"' for a in args) if args else ""
        full_command_line = f'{target_exe} {arg_str}'
        
        # Flush any old messages before restarting.
        _flush_queue(self.output_queue)
        
        logger.debug(">>> .create %s", full_command_line)
        # Use a sentinel to wait for the new process to be ready.
        sentinel = f"###CDB_RESTART_SENTINEL_{next(_SENTINEL_ID)}###"
        cmd = f".create {full_command_line}\n.echo {sentinel}\n"
        self.process.stdin.write(cmd)
        self.process.stdin.flush()

        try:
            # Wait specifically for the sentinel, ignoring any other prompts.
            # This ensures we wait until the new process is fully created and paused.
            output = self._wait_for(sentinel, timeout=15.0)
            
            # Sanity check to ensure the new process actually started.
            for line in output:
                if "Break instruction exception" in line:
                    logger.debug("New process created and paused at initial break.")
                    return True
            logger.error("Restart failed: Did not see initial break exception.")
            return False
            
        except TimeoutError:
            logger.error("Timed out waiting for process to restart.")
            return False


    def gN(self) -> None:
        """Issues a 'gN' (Go Not Handled) command to get past the initial break."""
        # We don't need to wait for a specific output here, just the next prompt.
        self.send_command("gN")




In [186]:
import os
name = "switch6_vs_stripped"
program = programs[name]
program.exe_path = os.path.abspath(program.exe_path)
print(f"{name}:")
print(f"  exe         = {program.exe_path}")
print(f"  cfr         = {program.cfr_path}")
print(f"  question    = {program.question!r}")
print(f"  file_offset = {hex(program.file_offset)}")
print(f"  rva         = {hex(program.rva)}")
print(f"  groundtruth = {[hex(x) for x in program.ground_truth]}\n")
print(f"  gt_rvas     = {program.gt_rvas}\n")


main_module_name = os.path.splitext(os.path.basename(program.exe_path))[0]
found_rvas = []

# The main loop iterates through the test cases
for i in range(1, len(program.gt_rvas) + 1):
    args = [str(n) for n in range(1, i + 1)]
    print(f"\n--- [TESTING] Arguments: {args} ---")

    try:
        # Create a BRAND NEW wrapper for each test case.
        # The 'with' statement ensures it's cleaned up automatically.
        with CDBWrapper() as cdb:
            # Start the debugger directly with the arguments for this run.
            if not cdb.start(program.exe_path, args=args):
                print("[ERROR] Failed to start CDB for this run.")
                continue

            # The process starts and is paused at the initial system break.
            # We don't need to call gN() or do any complex setup.

            # Get module info for this specific process instance.
            all_mods, all_vaddrs = cdb.lm()
            base_vaddr = ""
            module_name_for_bp = ""
            try:
                mod_index = -1
                for idx, mod in enumerate(all_mods):
                    if mod in main_module_name:
                        mod_index = idx
                        module_name_for_bp = mod
                        break
                if mod_index == -1: raise ValueError("Module not found")
                base_vaddr = all_vaddrs[mod_index]
                print(f"[INFO] Found module '{module_name_for_bp}' at base address {base_vaddr}")
            except (ValueError, IndexError):
                print(f"[ERROR] Could not find module '{main_module_name}'.")
                continue

            # Set the breakpoint and run.
            bp_address = f"{module_name_for_bp}+{hex(program.rva)}"
            cdb.bp(bp_address)
            print(f"[INFO] Breakpoint set at {bp_address}")

            # This g() will now work because it's in a clean session.
            cdb.g() 
            print(f"[SUCCESS] Hit breakpoint at {bp_address}")
            
            cdb.p() 
            
            regs = cdb.r()
            rip = regs.get("rip")

            if rip:
                base_val = int(base_vaddr.replace('`', ''), 16)
                rip_val = int(rip, 16)
                case_rva = hex(rip_val - base_val)
                print(f"  -> Case landed at RVA: {case_rva}")
                if case_rva not in found_rvas:
                    found_rvas.append(case_rva)

    except ProcessTerminationIntercepted:
        print("[INFO] Process terminated without hitting the breakpoint.")
        continue
    except TimeoutError:
        print("[WARN] Run timed out.")
        continue

print("\n--- Discovered RVAs ---")
print(sorted(found_rvas))
print("\n--- Ground Truth RVAs ---")
print(sorted(program.gt_rvas))

2025-08-06 22:33:52 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-06 22:33:52 [DEBUG] __main__: Starting debugger for 'c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe' with args=['1']
2025-08-06 22:33:52 [INFO] __main__: Debugger started, PID=6704
2025-08-06 22:33:52 [DEBUG] __main__: <<< 

2025-08-06 22:33:52 [DEBUG] __main__: >>> lm
2025-08-06 22:33:52 [DEBUG] __main__: <<< 

2025-08-06 22:33:52 [DEBUG] __main__: <<< >>>>>>>>>>>>> Waiting for Debugger Extensions Gallery to Initialize completed, duration 0.000 seconds

2025-08-06 22:33:52 [DEBUG] __main__: <<<    ----> Repository : UserExtensions, Enabled: true, Packages count: 0

2025-08-06 22:33:52 [DEBUG] __main__: <<<    ----> Repository : LocalInstalled, Enabled: true, Packages count: 29

2025-08-06 22:33:52 [DEBUG] __main__: <<< 

2025-08-06 22:33:52 [DEBUG] _

switch6_vs_stripped:
  exe         = c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe
  cfr         = ../sunbench25/benchmark/indirect_jumps/switch_statements/switch6_vs-stripped-cfr.json
  question    = "What are the file offsets for the instructions that are the targets of the 'jmp rax' instruction at file offset '0x6577' ?"
  file_offset = 0x6577
  rva         = 0x7177
  groundtruth = ['0x657b', '0x6584', '0x658d', '0x6596', '0x659f', '0x65a8', '0x65b1', '0x65ba', '0x65c3', '0x65cc', '0x65d5', '0x65de', '0x65e5']

  gt_rvas     = ['0x717b', '0x7184', '0x718d', '0x7196', '0x719f', '0x71a8', '0x71b1', '0x71ba', '0x71c3', '0x71cc', '0x71d5', '0x71de', '0x71e5']


--- [TESTING] Arguments: ['1'] ---


2025-08-06 22:33:52 [DEBUG] __main__: <<< 

2025-08-06 22:33:52 [DEBUG] __main__: <<< ************* Path validation summary **************

2025-08-06 22:33:52 [DEBUG] __main__: <<< Response                         Time (ms)     Location

2025-08-06 22:33:52 [DEBUG] __main__: <<< Deferred                                       srv*C:\symbols*https://msdl.microsoft.com/download/symbols

2025-08-06 22:33:52 [DEBUG] __main__: <<< Symbol search path is: srv*C:\symbols*https://msdl.microsoft.com/download/symbols

2025-08-06 22:33:52 [DEBUG] __main__: <<< Executable search path is: 

2025-08-06 22:33:52 [DEBUG] __main__: <<< ModLoad: 00007ff7`7c520000 00007ff7`7c5c1000   switch6.exe

2025-08-06 22:33:52 [DEBUG] __main__: <<< ModLoad: 00007ff8`5ab50000 00007ff8`5ad48000   ntdll.dll

2025-08-06 22:33:52 [DEBUG] __main__: <<< ModLoad: 00007ff8`59280000 00007ff8`59342000   C:\WINDOWS\System32\KERNEL32.DLL

2025-08-06 22:33:52 [DEBUG] __main__: <<< ModLoad: 00007ff8`58550000 00007ff8`58846000   C:

[INFO] Found module 'switch6' at base address 00007ff77c520000
[INFO] Breakpoint set at switch6+0x7177
[SUCCESS] Hit breakpoint at switch6+0x7177


2025-08-06 22:33:54 [DEBUG] __main__: <<< *** WARNING: Unable to verify checksum for switch6.exe

2025-08-06 22:33:54 [DEBUG] __main__: <<< switch6+0x7177:

2025-08-06 22:33:54 [DEBUG] __main__: <<< 00007ff7`7c527177 ffe0            jmp     rax {switch6+0x717b (00007ff7`7c52717b)}

2025-08-06 22:33:54 [DEBUG] __main__: <<< 0:000> switch6+0x717b:

2025-08-06 22:33:54 [DEBUG] __main__: <<< 00007ff7`7c52717b c704240c000000  mov     dword ptr [rsp],0Ch ss:0000000c`dc34fc10=7c59f110

2025-08-06 22:33:54 [DEBUG] __main__: <<< 0:000> ###CDB_SENTINEL_3###

2025-08-06 22:33:54 [DEBUG] __main__: >>> r
2025-08-06 22:33:54 [DEBUG] __main__: <<< 0:000> rax=00007ff77c52717b rbx=000001768a4b9ab0 rcx=00007ff77c520000

2025-08-06 22:33:54 [DEBUG] __main__: <<< rdx=000001768a4b9ab0 rsi=0000000000000000 rdi=000001768a4bdab0

2025-08-06 22:33:54 [DEBUG] __main__: <<< rip=00007ff77c52717b rsp=0000000cdc34fc10 rbp=0000000000000000

2025-08-06 22:33:54 [DEBUG] __main__: <<<  r8=000001768a4bdab0  r9=000001768

...


--- [TESTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'] ---


2025-08-06 22:36:03 [DEBUG] __main__: <<< 

2025-08-06 22:36:03 [DEBUG] __main__: <<< ************* Path validation summary **************

2025-08-06 22:36:03 [DEBUG] __main__: <<< Response                         Time (ms)     Location

2025-08-06 22:36:03 [DEBUG] __main__: <<< Deferred                                       srv*C:\symbols*https://msdl.microsoft.com/download/symbols

2025-08-06 22:36:03 [DEBUG] __main__: <<< Symbol search path is: srv*C:\symbols*https://msdl.microsoft.com/download/symbols

2025-08-06 22:36:03 [DEBUG] __main__: <<< Executable search path is: 

2025-08-06 22:36:03 [DEBUG] __main__: <<< ModLoad: 00007ff7`7c520000 00007ff7`7c5c1000   switch6.exe

2025-08-06 22:36:03 [DEBUG] __main__: <<< ModLoad: 00007ff8`5ab50000 00007ff8`5ad48000   ntdll.dll

2025-08-06 22:36:03 [DEBUG] __main__: <<< ModLoad: 00007ff8`59280000 00007ff8`59342000   C:\WINDOWS\System32\KERNEL32.DLL

2025-08-06 22:36:03 [DEBUG] __main__: <<< ModLoad: 00007ff8`58550000 00007ff8`58846000   C:

[INFO] Found module 'switch6' at base address 00007ff77c520000
[INFO] Breakpoint set at switch6+0x7177
[SUCCESS] Hit breakpoint at switch6+0x7177


2025-08-06 22:36:03 [DEBUG] __main__: <<< *** WARNING: Unable to verify checksum for switch6.exe

2025-08-06 22:36:03 [DEBUG] __main__: <<< switch6+0x7177:

2025-08-06 22:36:03 [DEBUG] __main__: <<< 00007ff7`7c527177 ffe0            jmp     rax {switch6+0x71e5 (00007ff7`7c5271e5)}

2025-08-06 22:36:03 [DEBUG] __main__: <<< 0:000> switch6+0x71e5:

2025-08-06 22:36:03 [DEBUG] __main__: <<< 00007ff7`7c5271e5 8b0424          mov     eax,dword ptr [rsp] ss:0000007a`c6f0fdb0=7c59f110

2025-08-06 22:36:03 [DEBUG] __main__: <<< 0:000> ###CDB_SENTINEL_63###

2025-08-06 22:36:03 [DEBUG] __main__: >>> r
2025-08-06 22:36:03 [DEBUG] __main__: <<< 0:000> rax=00007ff77c5271e5 rbx=00000214497a9b10 rcx=00007ff77c520000

2025-08-06 22:36:03 [DEBUG] __main__: <<< rdx=00000214497a9b10 rsi=0000000000000000 rdi=00000214497adb90

2025-08-06 22:36:03 [DEBUG] __main__: <<< rip=00007ff77c5271e5 rsp=0000007ac6f0fdb0 rbp=0000000000000000

2025-08-06 22:36:03 [DEBUG] __main__: <<<  r8=00000214497adb90  r9=00000214

  -> Case landed at RVA: 0x71e5

--- Discovered RVAs ---
['0x717b', '0x7184', '0x718d', '0x7196', '0x719f', '0x71e5']

--- Ground Truth RVAs ---
['0x717b', '0x7184', '0x718d', '0x7196', '0x719f', '0x71a8', '0x71b1', '0x71ba', '0x71c3', '0x71cc', '0x71d5', '0x71de', '0x71e5']


In [ ]:
def run_debug_session(args: List[str]) -> Optional[str]:
    """
    Runs a single, complete CDB session for a given set of arguments.
    Returns the discovered RVA on success, or None on failure.
    """
    print(f"--- [STARTING] Arguments: {args} ---")
    main_module_name = os.path.splitext(os.path.basename(program.exe_path))[0]
    
    try:
        # Each thread gets its own brand-new CDBWrapper instance.
        with CDBWrapper() as cdb:
            if not cdb.start(program.exe_path, args=args):
                print(f"[ERROR] Failed to start CDB for args: {args}")
                return None

            all_mods, all_vaddrs = cdb.lm()
            base_vaddr = ""
            module_name_for_bp = ""
            try:
                mod_index = -1
                for idx, mod in enumerate(all_mods):
                    if mod in main_module_name:
                        mod_index, module_name_for_bp = idx, mod
                        break
                if mod_index == -1: raise ValueError("Module not found")
                base_vaddr = all_vaddrs[mod_index]
            except (ValueError, IndexError):
                print(f"[ERROR] Could not find module for args: {args}")
                return None

            bp_address = f"{module_name_for_bp}+{hex(program.rva)}"
            cdb.bp(bp_address)

            cdb.g()
            
            cdb.p() 
            
            regs = cdb.r()
            rip = regs.get("rip")

            if rip:
                base_val = int(base_vaddr.replace('`', ''), 16)
                rip_val = int(rip, 16)
                case_rva = hex(rip_val - base_val)
                print(f"  -> [SUCCESS] Args {args} found RVA: {case_rva}")
                return case_rva

    except (ProcessTerminationIntercepted, TimeoutError):
        print(f"[INFO] Args {args} did not hit the breakpoint.")
        return None
    except Exception as e:
        print(f"[FATAL] An unexpected error occurred with args {args}: {e}")
        return None
    
import concurrent.futures


# 1. Prepare all the tasks in advance
all_arg_sets = [[str(n) for n in range(1, i + 1)] for i in range(1, 300)]
found_rvas = []

# 2. Create and manage the worker pool
# You can adjust max_workers based on your CPU cores (e.g., os.cpu_count())
# Start with 4-8 for a good balance.
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # The map function automatically runs the worker function on each item
    # in all_arg_sets and collects the results.
    results = executor.map(run_debug_session, all_arg_sets)

    # 3. Process the results after all tasks are complete
    for rva in results:
        if rva:
            found_rvas.append(rva)

print("\n--- Discovered RVAs ---")
print(sorted(list(set(found_rvas))))
print("\n--- Ground Truth RVAs ---")
print(sorted(program.gt_rvas))

2025-08-06 22:50:19 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-06 22:50:19 [DEBUG] __main__: Starting debugger for 'c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe' with args=['1']


2025-08-06 22:50:19 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-06 22:50:19 [DEBUG] __main__: Starting debugger for 'c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe' with args=['1', '2']
2025-08-06 22:50:19 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-06 22:50:19 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-06 22:50:19 [DEBUG] __main__: Starting debugger for 'c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe' with args=['1', '2', '3']
2025-08-06 22:50:19 [INFO] __main__: Debugger started, PID=37472
2025-08-06 22:50:19 [DEBUG] __main__: Initialized CDBWrapper with cdb_pa

--- [STARTING] Arguments: ['1'] ---
--- [STARTING] Arguments: ['1', '2'] ---
--- [STARTING] Arguments: ['1', '2', '3'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8'] ---


2025-08-06 22:50:20 [DEBUG] __main__: <<< Copyright (c) Microsoft Corporation. All rights reserved.

2025-08-06 22:50:20 [DEBUG] __main__: <<< 

2025-08-06 22:50:20 [DEBUG] __main__: <<< CommandLine: c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch6_vs-stripped.exe 1

2025-08-06 22:50:20 [DEBUG] __main__: <<< 

2025-08-06 22:50:20 [DEBUG] __main__: <<< >>>>>>>>>>>>> Waiting for Debugger Extensions Gallery to Initialize completed, duration 0.016 seconds

2025-08-06 22:50:20 [DEBUG] __main__: <<<    ----> Repository : UserExtensions, Enabled: true, Packages count: 0

2025-08-06 22:50:20 [DEBUG] __main__: <<<    ----> Repository : LocalInstalled, Enabled: true, Packages count: 29

2025-08-06 22:50:20 [DEBUG] __main__: <<< 

2025-08-06 22:50:20 [DEBUG] __main__: <<< Microsoft (R) Windows Debugger Version 10.0.26100.4654 AMD64

2025-08-06 22:50:20 [DEBUG] __main__: <<< Copyright (c) Microsoft Corporation. All rights

  -> [SUCCESS] Args ['1', '2', '3'] found RVA: 0x718d
  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8'] found RVA: 0x71e5
  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6'] found RVA: 0x71e5
  -> [SUCCESS] Args ['1', '2'] found RVA: 0x7184
  -> [SUCCESS] Args ['1'] found RVA: 0x717b
  -> [SUCCESS] Args ['1', '2', '3', '4', '5'] found RVA: 0x719f
  -> [SUCCESS] Args ['1', '2', '3', '4'] found RVA: 0x7196


...

--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9'] ------ [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] ---

--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] ---
--- [STARTING] Arguments: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'] ---


2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: >>> lm
2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: >>> lm
2025-08-06 22:50:31 [DEBUG] __main__: >>> lm
2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: <<< >>>>>>>>>>>>> Waiting for Debugger Extensions Gallery to Initialize completed, duration 0.015 seconds

2025-08-06 22:50:31 [DEBUG] __main__: <<<    ----> Repository : UserExtensions, Enabled: true, Packages count: 0

2025-08-06 22:50:31 [DEBUG] __main__: <<<    ----> Repository : LocalInstalled, Enabled: true, Packages count: 29

2025-08-06 22:50:31 [DEBUG] __main__: <<< 

2025-08-06 22:50:31 [DEBUG] __main__: <<< >>>>>>>>>>>>> Waiting for Debugger Extensions Gallery to Initialize completed, duration 0.015 seconds

2025-08-06 22:50:31 [DEBUG] __main__: 

  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] found RVA: 0x71e5
  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8', '9'] found RVA: 0x71e5
  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'] found RVA: 0x71e5
  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'] found RVA: 0x71e5


2025-08-06 22:50:31 [DEBUG] __main__: <<< *** WARNING: Unable to verify checksum for switch6.exe

2025-08-06 22:50:31 [DEBUG] __main__: <<< switch6+0x7177:

2025-08-06 22:50:31 [DEBUG] __main__: <<< 00007ff7`7c527177 ffe0            jmp     rax {switch6+0x71e5 (00007ff7`7c5271e5)}

2025-08-06 22:50:31 [DEBUG] __main__: <<< 0:000> switch6+0x71e5:

2025-08-06 22:50:31 [DEBUG] __main__: <<< 00007ff7`7c5271e5 8b0424          mov     eax,dword ptr [rsp] ss:000000cd`93d9f710=7c59f110

2025-08-06 22:50:31 [DEBUG] __main__: <<< 0:000> ###CDB_SENTINEL_119###

2025-08-06 22:50:31 [DEBUG] __main__: >>> r
2025-08-06 22:50:31 [DEBUG] __main__: <<< 0:000> rax=00007ff77c5271e5 rbx=000002843c009b10 rcx=00007ff77c520000

2025-08-06 22:50:31 [DEBUG] __main__: <<< rdx=000002843c009b10 rsi=0000000000000000 rdi=000002843c00db80

2025-08-06 22:50:31 [DEBUG] __main__: <<< rip=00007ff77c5271e5 rsp=000000cd93d9f710 rbp=0000000000000000

2025-08-06 22:50:31 [DEBUG] __main__: <<<  r8=000002843c00db80  r9=0000028

  -> [SUCCESS] Args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] found RVA: 0x71e5

--- Discovered RVAs ---
['0x717b', '0x7184', '0x718d', '0x7196', '0x719f', '0x71e5']

--- Ground Truth RVAs ---
['0x717b', '0x7184', '0x718d', '0x7196', '0x719f', '0x71a8', '0x71b1', '0x71ba', '0x71c3', '0x71cc', '0x71d5', '0x71de', '0x71e5']


In [ ]:
import concurrent.futures
from typing import Tuple
logging.basicConfig(level=logging.ERROR)
def run_debug_session(task: Tuple[Program, List[str]]) -> Optional[str]:
    """
    Runs a single, complete CDB session for a given program and set of arguments.
    Returns the discovered RVA on success, or None on failure.
    """
    # Unpack the task tuple to get the program and its arguments
    program, args = task
    
    # Use a print statement that identifies the program being tested
    print(f"--- [STARTING] {os.path.basename(program.exe_path)} with args: {args} ---")
    main_module_name = os.path.splitext(os.path.basename(program.exe_path))[0]
    
    try:
        with CDBWrapper() as cdb:
            if not cdb.start(program.exe_path, args=args):
                print(f"[ERROR] Failed to start CDB for {main_module_name} with args: {args}")
                return None

            all_mods, all_vaddrs = cdb.lm()
            base_vaddr = ""
            module_name_for_bp = ""
            try:
                mod_index = -1
                for idx, mod in enumerate(all_mods):
                    if mod in main_module_name:
                        mod_index, module_name_for_bp = idx, mod
                        break
                if mod_index == -1: raise ValueError("Module not found")
                base_vaddr = all_vaddrs[mod_index]
            except (ValueError, IndexError):
                print(f"[ERROR] Could not find module for {main_module_name} with args: {args}")
                return None

            bp_address = f"{module_name_for_bp}+{hex(program.rva)}"
            cdb.bp(bp_address)
            cdb.g()
            cdb.p()
            regs = cdb.r()
            rip = regs.get("rip")

            if rip:
                base_val = int(base_vaddr.replace('`', ''), 16)
                rip_val = int(rip, 16)
                case_rva = hex(rip_val - base_val)
                print(f"  -> [SUCCESS] {main_module_name} with args {args} found RVA: {case_rva}")
                return case_rva

    except (ProcessTerminationIntercepted, TimeoutError):
        print(f"[INFO] {main_module_name} with args {args} did not hit the breakpoint.")
        return None
    except Exception as e:
        print(f"[FATAL] An unexpected error occurred with {main_module_name} and args {args}: {e}")
        return None
    
# Set the number of parallel workers
MAX_WORKERS = os.cpu_count() or 4

# Outer loop to iterate through each program you want to test
for name, program in programs.items():
    if not "switch" in name:
        continue
    program.exe_path = os.path.abspath(program.exe_path)
    print(f"\n{'='*20} TESTING: {name} {'='*20}")
    print(f"  exe         = {program.exe_path}")
    print(f"  rva         = {hex(program.rva)}")
    print(f"  gt_rvas     = {program.gt_rvas}\n")

    # 1. Prepare all the tasks for the CURRENT program
    # The number of arguments to test can be heuristic, like len(gt_rvas) * 2
    max_args_to_test = 300
    all_arg_sets = [[str(n) for n in range(1, i + 1)] for i in range(1, max_args_to_test + 1)]
    
    # Create a list of (program, args) tuples
    tasks = [(program, arg_set) for arg_set in all_arg_sets]
    
    found_rvas = []

    # 2. Create and manage the worker pool for this program
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Map the worker function to the list of tasks
        results = executor.map(run_debug_session, tasks)

        # 3. Process the results after all tasks are complete
        for rva in results:
            if rva and rva not in found_rvas:
                found_rvas.append(rva)

    print(f"\n--- RESULTS for {name} ---")
    print("Discovered RVAs: ", sorted(found_rvas))
    print("Ground Truth RVAs:", sorted(program.gt_rvas))
    
    # Optional: Compare results for a pass/fail status
    if set(found_rvas) == set(program.gt_rvas):
        print("✅ STATUS: PASS")
    else:
        print("❌ STATUS: FAIL")

print(f"\n{'='*20} ALL TESTS COMPLETE {'='*20}")

2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-07 00:06:27 [DEBUG] __main__: Starting debugger for 'c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch4_vs-stripped.exe' with args=['1']
2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe
2025-08-07 00:06:27 [DEBUG] __main__: Initialized CDBWrapper with cdb_path=C:\Program Files (x86)\Wi


==================== TESTING: switch4_vs_stripped ====================
  exe         = c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch4_vs-stripped.exe
  rva         = 0x7170
  gt_rvas     = ['0x7172', '0x7174', '0x717d', '0x7186', '0x718f', '0x7198', '0x71a1', '0x71aa', '0x71b1']

--- [STARTING] switch4_vs-stripped.exe with args: ['1'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6', '7'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6', '7', '8'] ---
--- [STARTING] swit

2025-08-07 00:06:27 [DEBUG] __main__: <<< >>>>>>>>>>>>> Waiting for Debugger Extensions Gallery to Initialize completed, duration 0.031 seconds

2025-08-07 00:06:27 [DEBUG] __main__: <<<    ----> Repository : UserExtensions, Enabled: true, Packages count: 0

2025-08-07 00:06:27 [DEBUG] __main__: <<<    ----> Repository : LocalInstalled, Enabled: true, Packages count: 29

2025-08-07 00:06:27 [DEBUG] __main__: <<< 

2025-08-07 00:06:27 [DEBUG] __main__: <<< Microsoft (R) Windows Debugger Version 10.0.26100.4654 AMD64

2025-08-07 00:06:27 [DEBUG] __main__: <<< Copyright (c) Microsoft Corporation. All rights reserved.

2025-08-07 00:06:27 [DEBUG] __main__: <<< 

2025-08-07 00:06:27 [DEBUG] __main__: <<< CommandLine: c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch4_vs-stripped.exe 1

2025-08-07 00:06:27 [DEBUG] __main__: <<< 

2025-08-07 00:06:27 [DEBUG] __main__: <<< 

2025-08-07 00:06:27 [DEBUG] __main__: >>> lm


  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'] found RVA: 0x71b1
  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9'] found RVA: 0x717d
  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] found RVA: 0x71b1
  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] found RVA: 0x71b1


2025-08-07 00:06:38 [INFO] __main__: Stopping debugger
2025-08-07 00:06:38 [INFO] __main__: Stopping debugger
2025-08-07 00:06:38 [INFO] __main__: Stopping debugger
2025-08-07 00:06:38 [DEBUG] __main__: >>> q
2025-08-07 00:06:38 [DEBUG] __main__: >>> q
2025-08-07 00:06:38 [DEBUG] __main__: >>> q
2025-08-07 00:06:38 [DEBUG] __main__: <<< 0:000> quit:

2025-08-07 00:06:38 [DEBUG] __main__: <<< 0:000> quit:

2025-08-07 00:06:38 [DEBUG] __main__: <<< 0:000> quit:

2025-08-07 00:06:38 [DEBUG] __main__: <<< NatVis script unloaded from 'C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\Visualizers\atlmfc.natvis'

2025-08-07 00:06:38 [DEBUG] __main__: <<< NatVis script unloaded from 'C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\Visualizers\atlmfc.natvis'

2025-08-07 00:06:38 [DEBUG] __main__: <<< NatVis script unloaded from 'C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\Visualizers\atlmfc.natvis'

2025-08-07 00:06:38 [DEBUG] __main__: <<< NatVis script unloaded from 'C:\Progr

...

[INFO] switch4_vs-stripped with args ['1', '2', '3', '4'] did not hit the breakpoint.[INFO] switch4_vs-stripped with args ['1', '2'] did not hit the breakpoint.
[INFO] switch4_vs-stripped with args ['1', '2', '3'] did not hit the breakpoint.

--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'] ---
--- [STARTING] switch4_vs-stripped.exe with args: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'] ---


2025-08-07 00:06:48 [DEBUG] __main__: <<< Microsoft (R) Windows Debugger Version 10.0.26100.4654 AMD64

2025-08-07 00:06:48 [DEBUG] __main__: <<< Copyright (c) Microsoft Corporation. All rights reserved.

2025-08-07 00:06:48 [DEBUG] __main__: <<< Copyright (c) Microsoft Corporation. All rights reserved.

2025-08-07 00:06:48 [DEBUG] __main__: <<< 

2025-08-07 00:06:48 [DEBUG] __main__: <<< 

2025-08-07 00:06:48 [DEBUG] __main__: <<< CommandLine: c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch4_vs-stripped.exe 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22

2025-08-07 00:06:48 [DEBUG] __main__: <<< CommandLine: c:\Users\enya\Documents\proj\suns-dataset\control_flow_recovery\sunbench25\benchmark\indirect_jumps\switch_statements\switch4_vs-stripped.exe 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23

2025-08-07 00:06:48 [DEBUG] __main__: <<< 

2025-08-07 00:06:48 [DEBUG] __main__: <<< 

2025-

  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'] found RVA: 0x71b1
  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'] found RVA: 0x71b1


2025-08-07 00:06:49 [DEBUG] __main__: >>> r
2025-08-07 00:06:49 [DEBUG] __main__: <<< 0:000> rax=00007ff7e28f7174 rbx=00000186d15e9b50 rcx=00007ff7e28f0000

2025-08-07 00:06:49 [DEBUG] __main__: <<< rdx=00000186d15e9b50 rsi=0000000000000000 rdi=00000186d15edc30

2025-08-07 00:06:49 [DEBUG] __main__: <<< rip=00007ff7e28f7174 rsp=00000001000ffcc0 rbp=0000000000000000

2025-08-07 00:06:49 [DEBUG] __main__: <<<  r8=00000186d15edc30  r9=00000186d15ed5c0 r10=0000000000000000

2025-08-07 00:06:49 [DEBUG] __main__: <<< r11=00000001000ffcb0 r12=0000000000000000 r13=0000000000000000

2025-08-07 00:06:49 [DEBUG] __main__: <<< r14=0000000000000000 r15=0000000000000000

2025-08-07 00:06:49 [DEBUG] __main__: <<< iopl=0         nv up ei pl nz na po nc

2025-08-07 00:06:49 [DEBUG] __main__: <<< cs=0033  ss=002b  ds=002b  es=002b  fs=0053  gs=002b             efl=00000206

2025-08-07 00:06:49 [DEBUG] __main__: <<< switch4+0x7174:

2025-08-07 00:06:49 [DEBUG] __main__: <<< 00007ff7`e28f7174 c704240c0000

  -> [SUCCESS] switch4_vs-stripped with args ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'] found RVA: 0x7174


In [1]:
import pefile
import sys


def file_offset_to_virtual_address(pe_file_path, file_offset):
    try:
        # Load the PE file
        pe = pefile.PE(pe_file_path)

        # Get the base address, section alignment, and file alignment
        base_address = pe.OPTIONAL_HEADER.ImageBase
        section_alignment = pe.OPTIONAL_HEADER.SectionAlignment
        file_alignment = pe.OPTIONAL_HEADER.FileAlignment

        # Iterate through sections to find the one containing the file offset
        for section in pe.sections:
            # Get section details
            raw_data_offset = section.PointerToRawData
            raw_data_size = section.SizeOfRawData
            virtual_address = section.VirtualAddress

            # Check if the file offset falls within the section's raw data range
            if raw_data_offset <= file_offset < raw_data_offset + raw_data_size:
                # Calculate the offset within the section
                offset_within_section = file_offset - raw_data_offset

                # Calculate the virtual address
                final_address = base_address + virtual_address + offset_within_section
                print(
                    f"Let's compare! PEFile says {hex(pe.get_rva_from_offset(file_offset))}... I say {hex(final_address)}"
                )
                return final_address

        # If no section contains the file offset, return None
        return None

    except Exception as e:
        print(f"Error processing the PE file: {e}")
        return None


def virtual_address_to_file_offset(pe_file_path, virtual_address):
    try:
        # Load the PE file
        pe = pefile.PE(pe_file_path)

        # Get the base address, section alignment, and file alignment
        base_address = pe.OPTIONAL_HEADER.ImageBase
        section_alignment = pe.OPTIONAL_HEADER.SectionAlignment
        file_alignment = pe.OPTIONAL_HEADER.FileAlignment

        # Calculate the relative virtual address (RVA)
        rva = virtual_address - base_address

        # Iterate through sections to find the one containing the RVA
        for section in pe.sections:
            # Get section details
            raw_data_offset = section.PointerToRawData
            raw_data_size = section.SizeOfRawData
            virtual_address_start = section.VirtualAddress
            virtual_address_end = virtual_address_start + section.Misc_VirtualSize

            # Check if the RVA falls within the section's virtual address range
            if virtual_address_start <= rva < virtual_address_end:
                # Calculate the offset within the section
                offset_within_section = rva - virtual_address_start

                # Calculate the file offset
                file_offset = raw_data_offset + offset_within_section
                print(
                    f"Let's compare! PEFile says {pe.get_offset_from_rva(rva)}... I say {file_offset}"
                )
                return file_offset

        # If no section contains the RVA, return None
        return None

    except Exception as e:
        print(f"Error processing the PE file: {e}")
        return None


executables = {
    "anon_call2_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe",
    "anon_call3_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe",
    "virtual7_vs_stripped": "../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe",
    "anon_jump1_vs_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe",
    "anon_jump1_vs_2_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe",
    "switch4_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe",
    "switch5_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe",
    "switch6_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch6_vs-stripped.exe",
}

cfr_mapping = {
    path: path.replace(".exe", "-cfr.json") for key, path in executables.items()
}
import json
import re

for exepath, cfrpath in cfr_mapping.items():
    with open(cfrpath, "r") as f:
        cfr = json.load(f)
    question = cfr["question"]
    match = re.search(r"'([^']*)'(?!.*')", question)
    offset = match.group(1)
    print(f"{exepath}: {offset}")
    file_offset_to_virtual_address(exepath, int(offset, 16))

    import json
import re
import pefile
import os
from typing import List, Dict
from dataclasses import dataclass


@dataclass
class Program:
    exe_path: str
    cfr_path: str
    question: str
    file_offset: int  # offset inside file (from the question)
    rva: int  # runtime RVA (for breakpoints)
    ground_truth: List[int]  # list of file offsets from CFR JSON
    gt_rvas: List[int]  # list of RVA of GT from CFR JSON


# original flat mapping
executables: Dict[str, str] = {
    "anon_call2_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe",
    "anon_call3_vs_stripped": "../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe",
    "virtual7_vs_stripped": "../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe",
    "anon_jump1_vs_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe",
    "anon_jump1_vs_2_stripped": "../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe",
    "switch4_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe",
    "switch5_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe",
    "switch6_vs_stripped": "../sunbench25/benchmark/indirect_jumps/switch_statements/switch6_vs-stripped.exe",
}

programs: Dict[str, Program] = {}

for name, exe_path in executables.items():
    # derive the CFR JSON path
    cfr_path = exe_path.replace(".exe", "-cfr.json")

    # load the question, extract the quoted offset, and ground truth
    with open(cfr_path, "r") as f:
        cfr = json.load(f)

    question = cfr.get("question", "")
    m = re.search(r"'([^']*)'(?!.*')", question)
    if not m:
        raise ValueError(f"Could not parse offset from question for {name!r}")
    file_offset = int(m.group(1), 16)

    # parse ground truth offsets
    gt_strs = cfr.get("groundtruth", [])
    ground_truth = [int(off, 16) for off in gt_strs]

    # compute the RVA from file offset
    pe = pefile.PE(exe_path)
    rva = pe.get_rva_from_offset(file_offset)

    gt_rvas = [hex(pe.get_rva_from_offset(gt)) for gt in ground_truth]

    # store everything in our nice dict
    programs[name] = Program(
        exe_path=exe_path,
        cfr_path=cfr_path,
        question=question,
        file_offset=file_offset,
        rva=rva,
        ground_truth=ground_truth,
        gt_rvas=gt_rvas,
    )


import subprocess
import threading
import queue
import time
import os
import logging
import re
import itertools
from typing import Optional, Callable, List, Dict, Tuple

logging.basicConfig(
    level=logging.ERROR,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)


class ProcessTerminationIntercepted(Exception):
    """Raised when debugger hits NtTerminateProcess — process is about to terminate."""

    pass


_PROMPT_RE = re.compile(r"\d+:\d+\s*> ?$")  # e.g. “0:000> ” or “12:34> ”
_SENTINEL_ID = itertools.count(1)  # 1,2,3… — unique per command


def _flush_queue(q: queue.Queue) -> None:
    """Remove every line currently waiting in *q* (non-blocking)."""
    try:
        while True:
            q.get_nowait()
    except queue.Empty:
        pass


class CDBWrapper:
    """
    A thin, logging-centric wrapper around Microsoft CDB/WinDbg.
    Uses a per-command sentinel so every helper blocks until the *next*
    debugger prompt – no more manual `wait_for_prompt()` juggling.
    """

    def __init__(
        self, cdb_path: str | None = None, symbol_cache_path: str | None = r"C:\symbols"
    ) -> None:
        self.cdb_path = (
            cdb_path or r"C:\Program Files (x86)\Windows Kits\10\Debuggers\x64\cdb.exe"
        )
        self.symbol_cache_path = symbol_cache_path
        self.process: Optional[subprocess.Popen] = None
        self.process: Optional[subprocess.Popen] = None
        self.output_queue = queue.Queue()
        self.output_thread: Optional[threading.Thread] = None
        self.error_thread: Optional[threading.Thread] = None
        self.running = False
        self.output_callback: Optional[Callable[[str], None]] = None
        self.target_exe = None

        logger.debug("Initialized CDBWrapper with cdb_path=%s", self.cdb_path)

    # PROC LIFECYCLE
    def start(
        self,
        target_exe: str,
        args: List[str] | None = None,
        output_callback: Callable[[str], None] | None = None,
    ) -> bool:
        """
        Launch *target_exe* under CDB.  Returns **True** on success.
        Subsequent helpers block until the debugger prompt re-appears.
        """
        if self.running:
            logger.warning("Debugger is already running")
            return False

        symbol_path = (
            f"srv*{self.symbol_cache_path}*https://msdl.microsoft.com/download/symbols"
        )

        self.output_callback = output_callback
        self.target_exe = target_exe
        logger.debug("Starting debugger for '%s' with args=%s", target_exe, args)

        cmd = [
            self.cdb_path,
            "-lines",
            "-y",
            symbol_path,
            "-srcpath",
            os.path.dirname(target_exe),
            target_exe,
        ]
        if args:
            cmd.extend(args)

        try:
            self.process = subprocess.Popen(
                cmd,
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1,  # line-buffered
            )
            self.running = True

            self.output_thread = threading.Thread(
                target=self._drain, args=(self.process.stdout,)
            )
            self.error_thread = threading.Thread(
                target=self._drain, args=(self.process.stderr,)
            )
            for t in (self.output_thread, self.error_thread):
                t.daemon = True
                t.start()

            logger.info("Debugger started, PID=%s", self.process.pid)
            # initial loader prompt:
            self._wait_for_prompt()
            return True

        except Exception:
            logger.exception("Failed to start debugger")
            return False

    def attach(
        self, pid: int, output_callback: Callable[[str], None] | None = None
    ) -> bool:
        """Attach to a running process."""
        if self.running:
            logger.warning("Debugger is already running")
            return False

        self.output_callback = output_callback
        cmd = [self.cdb_path, "-p", str(pid)]

        try:
            self.process = subprocess.Popen(
                cmd,
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1,
            )
            self.running = True
            self.output_thread = threading.Thread(
                target=self._drain, args=(self.process.stdout,), daemon=True
            )
            self.error_thread = threading.Thread(
                target=self._drain, args=(self.process.stderr,), daemon=True
            )
            self.output_thread.start()
            self.error_thread.start()

            logger.info("Attached to PID=%s", pid)
            self._wait_for_prompt()
            return True

        except Exception:
            logger.exception("Failed to attach debugger")
            return False

    def stop(self) -> None:
        """Gracefully quit the debugger."""
        if not self.running:
            return

        logger.info("Stopping debugger")
        try:
            self.send_command("q", wait=True)
        except Exception:
            pass

        self.running = False
        if self.process and self.process.poll() is None:
            self.process.terminate()
            self.process.wait(timeout=5)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop()

    # IO
    def _drain(self, stream) -> None:
        """Background reader: push every debugger line into *self.output_queue*."""
        while True:
            try:
                line = stream.readline()
                if not line:
                    break
                self.output_queue.put(line)
                if self.output_callback:
                    self.output_callback(line)
            except Exception:
                break

    def send_command(self, command: str, *, wait: bool = True) -> List[str] | None:
        """
        Send *command* to CDB.  If *wait* (default) is True, block until the
        debugger prints the next prompt and return **all** captured lines.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        sentinel = f"###CDB_SENTINEL_{next(_SENTINEL_ID)}###"

        _flush_queue(self.output_queue)  # purge stale lines
        full_cmd = f"{command}\n.echo {sentinel}\n"
        logger.debug(">>> %s", command)
        self.process.stdin.write(full_cmd)
        self.process.stdin.flush()

        return self._wait_for(sentinel) if wait else None

    def _wait_for(self, sentinel: str, timeout: float = 10.0) -> List[str]:
        """Internal: block until *sentinel* (or fresh prompt) is seen."""
        deadline = time.time() + timeout
        collected: List[str] = []

        while time.time() < deadline:
            try:
                line = self.output_queue.get(timeout=0.25)
                logger.debug(f"<<< {line}")
                collected.append(line)
                if sentinel in line or _PROMPT_RE.search(line.rstrip()):
                    return collected
            except queue.Empty:
                pass

        raise TimeoutError("Timed out waiting for debugger prompt")

    def _wait_for_prompt(self, timeout: float = 10.0) -> None:
        """Convenience wrapper when no sentinel was used (e.g. after start)."""
        _ = self._wait_for("", timeout=timeout)  # empty sentinel → wait for prompt only

    # CONVENIENCE
    def lm(self) -> Tuple[List[str], List[str]]:
        """Return a list of loaded module names and their virtual addresses."""
        lines = self.send_command("lm")
        mods = []
        vaddrs = []
        # Regex to reliably capture start address and module name from 'lm' output.
        mod_re = re.compile(r"^\s*([0-9a-fA-F`]+)\s+([0-9a-fA-F`]+)\s+([^\s]+)")

        if not lines:
            logger.warning("`lm` command returned no output.")
            return [], []

        for ln in lines:
            match = mod_re.match(ln)
            if match:
                # group(1) is the start address, e.g., "00007ff6`e4b40000"
                # group(2) is the module name, e.g., "switch6_vs-stripped"
                vaddrs.append(match.group(1).replace("`", ""))
                mods.append(match.group(3))

        logger.debug("Loaded modules: %s", mods)
        logger.debug("Module vaddrs: %s", vaddrs)
        return mods, vaddrs

    def bp(self, address: str) -> None:
        """Set breakpoint at *address* (RVA or absolute)."""
        self.send_command(f"bp {address}")

    def g(self, timeout: float = 10.0) -> str:
        """
        Run until next event; returns 'BREAKPOINT' or 'TERMINATED'.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        _flush_queue(self.output_queue)
        logger.debug(">>> g (waiting for BP or Terminate)")
        self.process.stdin.write("g\n")
        self.process.stdin.flush()

        deadline = time.time() + timeout
        while time.time() < deadline:
            try:
                line = self.output_queue.get(timeout=0.25)
                logger.debug(f"<<< {line}")
                # Check for the two events we care about
                if "Breakpoint" in line and "hit" in line:
                    return "BREAKPOINT"
                if "ntdll!NtTerminateProcess" in line:
                    raise ProcessTerminationIntercepted("Process terminated during 'g'")
            except queue.Empty:
                # If the process died without a clean exit message, poll() will catch it.
                if self.process.poll() is not None:
                    raise ProcessTerminationIntercepted(
                        "Process terminated unexpectedly"
                    )
                continue

        raise TimeoutError("Timed out waiting for debugger event after 'g'")

    def t(self) -> None:
        """Single-step one instruction."""
        self.send_command("t")

    def p(self) -> None:
        """Single-step over one instruction."""
        self.send_command("p")

    def r(self) -> Dict[str, str]:
        """Return a dict of current register values."""
        out = self.send_command("r")
        regs: Dict[str, str] = {}
        for ln in out:
            for tok in ln.strip().split():
                if "=" in tok:
                    k, v = tok.split("=", 1)
                    regs[k] = v
        return regs

    def setargs(self, args: List[str]) -> None:
        """Queue new CLI args with `.create`."""
        arg_str = " ".join(f'"{a}"' for a in args)
        self.send_command(f".create {self.target_exe} {arg_str}")

    def restart(self, target_exe: str, args: List[str] | None = None) -> bool:
        """
        Terminates the current debuggee and starts a new one with new arguments,
        reusing the existing CDB session for speed.
        """
        if not self.running or not self.process:
            raise RuntimeError("Debugger not running")

        arg_str = " ".join(f'"{a}"' for a in args) if args else ""
        full_command_line = f"{target_exe} {arg_str}"

        # Flush any old messages before restarting.
        _flush_queue(self.output_queue)

        logger.debug(">>> .create %s", full_command_line)
        # Use a sentinel to wait for the new process to be ready.
        sentinel = f"###CDB_RESTART_SENTINEL_{next(_SENTINEL_ID)}###"
        cmd = f".create {full_command_line}\n.echo {sentinel}\n"
        self.process.stdin.write(cmd)
        self.process.stdin.flush()

        try:
            # Wait specifically for the sentinel, ignoring any other prompts.
            # This ensures we wait until the new process is fully created and paused.
            output = self._wait_for(sentinel, timeout=15.0)

            # Sanity check to ensure the new process actually started.
            for line in output:
                if "Break instruction exception" in line:
                    logger.debug("New process created and paused at initial break.")
                    return True
            logger.error("Restart failed: Did not see initial break exception.")
            return False

        except TimeoutError:
            logger.error("Timed out waiting for process to restart.")
            return False

    def gN(self) -> None:
        """Issues a 'gN' (Go Not Handled) command to get past the initial break."""
        # We don't need to wait for a specific output here, just the next prompt.
        self.send_command("gN")


import concurrent.futures
from typing import Tuple
from tabulate import tabulate


def run_debug_session(task: Tuple[Program, List[str]]) -> Optional[str]:
    """
    Runs a single, complete CDB session for a given program and set of arguments.
    Returns the discovered RVA on success, or None on failure.
    """
    # Unpack the task tuple to get the program and its arguments
    program, args = task

    # Use a print statement that identifies the program being tested
    print(f"--- [STARTING] {os.path.basename(program.exe_path)} with args: {args} ---")
    main_module_name = os.path.splitext(os.path.basename(program.exe_path))[0]

    try:
        with CDBWrapper() as cdb:
            if not cdb.start(program.exe_path, args=args):
                print(
                    f"[ERROR] Failed to start CDB for {main_module_name} with args: {args}"
                )
                return None

            all_mods, all_vaddrs = cdb.lm()
            base_vaddr = ""
            module_name_for_bp = ""
            try:
                mod_index = -1
                for idx, mod in enumerate(all_mods):
                    if mod in main_module_name:
                        mod_index, module_name_for_bp = idx, mod
                        break
                if mod_index == -1:
                    raise ValueError("Module not found")
                base_vaddr = all_vaddrs[mod_index]
            except (ValueError, IndexError):
                print(
                    f"[ERROR] Could not find module for {main_module_name} with args: {args}"
                )
                return None

            bp_address = f"{module_name_for_bp}+{hex(program.rva)}"
            cdb.bp(bp_address)
            cdb.g()
            cdb.p()
            regs = cdb.r()
            rip = regs.get("rip")

            if rip:
                base_val = int(base_vaddr.replace("`", ""), 16)
                rip_val = int(rip, 16)
                case_rva = hex(rip_val - base_val)
                print(
                    f"  -> [SUCCESS] {main_module_name} with args {args} found RVA: {case_rva}"
                )
                return case_rva

    except (ProcessTerminationIntercepted, TimeoutError):
        print(f"[INFO] {main_module_name} with args {args} did not hit the breakpoint.")
        return None
    except Exception as e:
        print(
            f"[FATAL] An unexpected error occurred with {main_module_name} and args {args}: {e}"
        )
        return None

############################### MAIN ##############################

# Set the number of parallel workers
MAX_WORKERS = os.cpu_count() or 4

summary = []  # Will hold results for the final table

# Outer loop to iterate through each program you want to test
for name, program in programs.items():
    if "switch" not in name:
        continue
    program.exe_path = os.path.abspath(program.exe_path)
    print(f"\n{'=' * 20} TESTING: {name} {'=' * 20}")
    print(f"  exe         = {program.exe_path}")
    print(f"  rva         = {hex(program.rva)}")
    print(f"  gt_rvas     = {program.gt_rvas}\n")

    max_args_to_test = 300
    all_arg_sets = [
        [str(n) for n in range(2, i + 1)] for i in range(1, max_args_to_test + 1)
    ]
    all_arg_sets.insert(0, "")  # include argc = 1

    tasks = [(program, arg_set) for arg_set in all_arg_sets]
    found_rvas = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = executor.map(run_debug_session, tasks)
        for rva in results:
            if rva and rva not in found_rvas:
                found_rvas.append(rva)

    found_rvas_sorted = sorted(found_rvas)
    gt_rvas_sorted = sorted(program.gt_rvas)
    status = "PASS" if set(found_rvas) == set(program.gt_rvas) else "FAIL"

    print(f"\n--- RESULTS for {name} ---")
    print("Discovered RVAs: ", found_rvas_sorted)
    print("Ground Truth RVAs:", gt_rvas_sorted)
    print(f"{'✅' if status == 'PASS' else '❌'} STATUS: {status}")

    # Save result for final summary table
    summary.append({
        "Program": name,
        "Expected": ", ".join(r for r in gt_rvas_sorted),
        "Found": ", ".join(r for r in found_rvas_sorted),
        "Status": status
    })

print(f"\n{'=' * 20} ALL TESTS COMPLETE {'=' * 20}")

# Print final table
print("\nFinal Summary Table:\n")
print(tabulate(summary, headers="keys", tablefmt="fancy_grid"))

../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call2_vs-stripped.exe: 0x5d1
Let's compare! PEFile says 0x11d1... I say 0x4011d1
../sunbench25/benchmark/indirect_calls/anonymous_functions/anon_call3_vs-stripped.exe: 0x3f4
Let's compare! PEFile says 0x11f4... I say 0x4011f4
../sunbench25/benchmark/indirect_calls/virtual_tables/virtual7_vs-stripped.exe: 0x66c0
Let's compare! PEFile says 0x72c0... I say 0x1400072c0
../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs-stripped.exe: 0x43e
Let's compare! PEFile says 0x103e... I say 0x40103e
../sunbench25/benchmark/indirect_jumps/anonymous_functions/anon_jump1_vs_2-stripped.exe: 0x43e
Let's compare! PEFile says 0x103e... I say 0x40103e
../sunbench25/benchmark/indirect_jumps/switch_statements/switch4_vs-stripped.exe: 0x6570
Let's compare! PEFile says 0x7170... I say 0x140007170
../sunbench25/benchmark/indirect_jumps/switch_statements/switch5_vs-stripped.exe: 0x6568
Let's compare! PEFile says 0x7168... I say 